# Parte de definición y preparación de entorno
### Solo se ejecuta una vez al inicio



In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
root_path = '/content/drive/My Drive/ModeloKeras/Test/'

In [0]:
import os
os.chdir(root_path)

In [0]:
!pip install -q pydub

In [0]:
# Load python libraries
%matplotlib inline
import numpy as np
import pandas as pd
import random
from scipy.io import wavfile
from sklearn.preprocessing import scale
import librosa.display
import librosa
import matplotlib.pyplot as plt
import os
import concurrent
from tqdm import tqdm
# Load audio libraries
from pydub import AudioSegment
import math
# Load tensorflow libraries
from tensorflow.keras.models import model_from_json
from tensorflow.keras.applications.densenet import DenseNet201, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from time import time
import numpy as np
import json

In [0]:
# Directories
audio_dir='/content/drive/My Drive/ModeloKeras/Test/audio'
chunk_dir ='/content/drive/My Drive/ModeloKeras/Test/chunk/'
testing_dir = '/content/drive/My Drive/ModeloKeras/Test/melspectrograms/'

In [0]:
# Function definition to save melspectrograms in the defined directory
def save_melspectrogram(directory_path, file_name, sampling_rate=44100):
    """ Will save spectogram into current directory"""
    
    path_to_file = os.path.join(directory_path, file_name)
    data, sr = librosa.load(path_to_file, sr=sampling_rate, mono=True)
    data = scale(data)

    melspec = librosa.feature.melspectrogram(y=data, sr=sr, n_mels=128)
    # Convert to log scale (dB) using the peak power (max) as reference
        # per suggestion from Librbosa: https://librosa.github.io/librosa/generated/librosa.feature.melspectrogram.html
    log_melspec = librosa.power_to_db(melspec, ref=np.max)  
    librosa.display.specshow(log_melspec, sr=sr)
    
    # create saving directory
    directory = '/content/drive/My Drive/ModeloKeras/Test/melspectrograms/'
    os.makedirs(directory, exist_ok=True)
    
    plt.savefig(directory + '/' + file_name.strip('.wav') + '.png')

In [0]:
# Function definition to compute the melspectrograms for each audio chunk
def compute_and_save_mel_spectogram(data):
    directory_path, filename = data
    save_melspectrogram(directory_path, filename, sampling_rate=44100)
    return filename

In [0]:
# Function definition to obtain the top3-predictions array for each audio chunk
def get_top_k_predictions(preds, label_map, k=5, print_flag=False):
    sorted_array = np.argsort(preds)[::-1]
    top_k = sorted_array[:k]
    label_map_flip = dict((v,k) for k,v in label_map.items())
    
    y_pred = []
    for label_index in top_k:
        if print_flag:
            print("{} ({})".format(label_map_flip[label_index], preds[label_index]))
        y_pred.append(label_map_flip[label_index])
        
    return y_pred

In [10]:
# load json and create model
json_file = open('/content/drive/My Drive/ModeloKeras/DenseNet201V5_model_21_frozen_layers.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/My Drive/ModeloKeras/MixV1_New_Densenet201_stft_weights.best.hdf5")
print("Loaded model from disk")

Loaded model from disk


In [0]:
label_map= {'Applause': 0, 'Bark': 1, 'Burping_or_eructation': 2, 'Computer_keyboard': 3, 'Cough': 4, 'Drawer_open_or_close': 5, 'Fart': 6, 'Finger_snapping': 7, 'Knock': 8, 'Laughter': 9, 'Meow': 10, 'Silence': 11, 'Telephone': 12, 'breathing': 13, 'brushing_teeth': 14, 'can_opening': 15, 'crying_baby': 16, 'drinking_sipping': 17, 'footsteps': 18, 'glass_breaking': 19, 'pouring_water': 20, 'sneezing': 21, 'snoring': 22, 'toilet_flush': 23}

# Parte de ejecución 
### Debe ejecutarse cada vez que reconozca que hay un nuevo archivo de audio

In [0]:
# Obtain the name of the recorded audio
audio = os.listdir(audio_dir + '/')
audio_name = ' '.join(map(str, audio))

In [0]:
# Obtain the duration of the recorded audio
os.chdir(audio_dir)
audio = AudioSegment.from_file(audio_name)
duracion = round(audio.duration_seconds,0)

In [0]:
# Obtain the recording datetime of the audio
import time
import datetime as dt
creation = time.ctime(os.path.getctime(audio_name))
date_time_obj = dt.datetime.strptime(creation, "%a %b %d %H:%M:%S %Y")
date_time_obj = date_time_obj + dt.timedelta(hours=2) # UTC +02:00 (Europe/Madrid)

rec_date_time = date_time_obj - dt.timedelta(0,seconds=duracion)
rec_date = rec_date_time.date()
rec_time = rec_date_time.time()

In [0]:
# Create the windowing
# Pydub works in milliseconds
s_vent = 4
ventana = 4 * 1000
i = 0
x = (duracion - s_vent) * 1000
fin = math.ceil(x)

# Create chunks saving directory
os.makedirs(chunk_dir, exist_ok=True)

# Audio splitting
lista = list(range(i, fin, 2000))
for i in lista:
  chunk = audio[i:ventana + i]
  chunk.export("/content/drive/My Drive/ModeloKeras/Test/chunk/{}-{}.wav".format(audio_name[:-4],int(i/1000)), format="wav")
  i += 2000

In [42]:
# Create the melspectrograms for each audio chunk
results = []

file_list = os.listdir(chunk_dir)
for audio_files in file_list:
    n_audio_files = len(audio_files)
    directory_path = chunk_dir
    data = list(zip([directory_path]*n_audio_files, [audio_files]))
        # Create a pool of processes. By default, one is created for each CPU in your machine.
    with concurrent.futures.ProcessPoolExecutor() as executor:
            # Process the list of files, but split the work across the process pool to use all CPUs!
        for out_fn in tqdm(executor.map(compute_and_save_mel_spectogram, data), total=len(data)):
                results.append(out_fn)
        

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:190: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
100%|██████████| 1/1 [00:01<00:00,  1.25s/it]


In [0]:
# Use the model to predict and classificate
y_pred = []
file_list = os.listdir(testing_dir)
for file_name in file_list:
    img_path = testing_dir +  '/' + file_name
        
    img = image.load_img(img_path, target_size=(224, 224))
        
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)* 1./255
        
    preds = loaded_model.predict(x)[0]
        
    y_pred.append(get_top_k_predictions(preds, label_map, k=2)[0:2])

df = pd.DataFrame({'File':file_list,'Pred':y_pred})

In [0]:
# Clear all the audios and melspectrograms
import shutil
shutil.rmtree('/content/drive/My Drive/ModeloKeras/Test/chunk/')
shutil.rmtree('/content/drive/My Drive/ModeloKeras/Test/melspectrograms/')
#os.remove(audio_dir + '/' + audio_name)

In [0]:
# Create the final vector (it indicates cough detection) and the times vector
x = 0
lista = list(range(x, len(df['File']), 1))
vector = [0] * len(df['File'])
times_vector = [0] * len(df['File'])
date_vector = [0] * len(df['File'])

for x in lista:
  if df["Pred"][x][0] == 'Cough' or df["Pred"][x][1] == "Cough":
    vector[x] = 1
  else:
    vector[x] = 0
  times_vector[x] = str((rec_date_time + dt.timedelta(0,seconds=x*2)).time())
  date_vector[x] = str((rec_date_time + dt.timedelta(0,seconds=x*2)).date())

In [46]:
df = pd.DataFrame({'File':file_list, 'Date':date_vector, 'Time':times_vector,'Pred':vector})
print(df)

                        File        Date      Time  Pred
0    Copia de audio123-0.png  2020-05-08  21:27:06     0
1    Copia de audio123-2.png  2020-05-08  21:27:08     0
2    Copia de audio123-4.png  2020-05-08  21:27:10     0
3    Copia de audio123-6.png  2020-05-08  21:27:12     0
4    Copia de audio123-8.png  2020-05-08  21:27:14     0
5   Copia de audio123-10.png  2020-05-08  21:27:16     0
6   Copia de audio123-12.png  2020-05-08  21:27:18     0
7   Copia de audio123-14.png  2020-05-08  21:27:20     0
8   Copia de audio123-16.png  2020-05-08  21:27:22     0
9   Copia de audio123-18.png  2020-05-08  21:27:24     1
10  Copia de audio123-20.png  2020-05-08  21:27:26     1
11  Copia de audio123-22.png  2020-05-08  21:27:28     0
12  Copia de audio123-24.png  2020-05-08  21:27:30     0
13  Copia de audio123-26.png  2020-05-08  21:27:32     1
14  Copia de audio123-28.png  2020-05-08  21:27:34     0
15  Copia de audio123-30.png  2020-05-08  21:27:36     1
16  Copia de audio123-32.png  2

In [47]:
df.to_gbq('dp4.audio', project_id='vertical-realm-252209', if_exists='append')

1it [00:06,  6.55s/it]
